In [1]:
import os
import numpy as np
import pickle
from qqqhe_utils import param_to_data, data_to_tree

rng = np.random.default_rng()


In [2]:
n = 100   # number of variables
Vb = 100  # maximum upper bound of variables

d = list(rng.integers(low=1, high=Vb+1, size=n))
Xv = [rng.integers(low=1, high=d[i]+1) for i in range(n)]
Xw = [rng.integers(low=1, high=d[i]+1) for i in range(n)]
a = list(rng.random(n))
b = list(rng.uniform(low=-1, high=1, size=n))

for obj in ['f', 'crash', 'fuel']:
    data = param_to_data(obj, d, Xv, Xw, a, b)
    R = data['ubNested'][n-1]

    for alpha in [0.1, 0.2, 0.5, 1, 2, 5, 10]:
        new_directory_path = f'data/{obj}-n{n}/o{alpha}'
        if not os.path.exists(new_directory_path):
            os.makedirs(new_directory_path)

        np.savez(f'{new_directory_path}/params.npz', n=n, R=R)

        for t in range(1000):
            dt = [int(np.clip(np.rint(rng.normal(d[i], alpha)), 1, Vb)) for i in range(n)]
            Xvt = [int(np.clip(np.rint(rng.normal(Xv[i], alpha)), 1, dt[i])) for i in range(n)]
            Xwt = [int(np.clip(np.rint(rng.normal(Xw[i], alpha)), 1, dt[i])) for i in range(n)]

            at = [max(0, rng.normal(a[i], 0.01 * alpha)) for i in range(n)]
            bt = [rng.normal(b[i], 0.01 * alpha) for i in range(n)]

            data = param_to_data(obj, dt, Xvt, Xwt, at, bt)
            Gt = data_to_tree(data)

            with open(f'{new_directory_path}/{t}.pkl', 'wb') as f:
                pickle.dump(Gt, f)